In [0]:
#import requests
#import time
#import json
#import sys
#import threading
#import queue
#from datetime import timedelta

In [1]:
!ls -lh

total 4.0K
drwxr-xr-x 2 root root 4.0K Oct 18 16:40 sample_data


In [0]:
!mkdir themoviedb_people

In [76]:
!ls /content -lh

total 1.5G
-rw-r--r-- 1 root root 2.5K Sep 23 14:25 adc.json
-rw-r--r-- 1 root root 1.5G Sep 23 13:43 movies.zip
drwxr-xr-x 2 root root 4.0K Sep 20 00:09 sample_data
drwxr-xr-x 3 root root 4.0K Sep 23 13:46 themoviedb_data
drwxrwxrwx 2 1000 1000 4.0K Sep 23 11:59 trial
drwxr-xr-x 3 root root 4.0K Sep 23 13:06 trial_files
-rw-r--r-- 1 root root 580K Sep 23 14:25 trial.tar
-rw-r--r-- 1 root root 126K Sep 23 14:25 trial.tar.bz2
-rw-r--r-- 1 root root 176K Sep 23 13:03 trial.zip


In [0]:
!ls themoviedb_people

In [0]:
!ls /content/themoviedb_people

In [0]:
#DIR = "/content/themoviedb_people"
DIR = r"C:\Users\Santiago Rubio\Documents\201803 VII Curso Data Science\TFM\Data\themoviedb_data\trial" 

In [0]:
#!rm /content/themoviedb_people/*

In [0]:
#Download to local
from google.colab import files

with open('example.txt', 'w') as f:
  f.write('some content')

files.download('example.txt')

In [0]:
#Download to local
from google.colab import files

with open('544200.json', 'w') as f:
  f.write(DIR+'/544200.json')

files.download('544200.json')

In [0]:
#Import from local
from google.colab import files

In [0]:
files.upload?

In [0]:
uploaded = files.upload()

Saving 2.json to 2.json


In [0]:
uploaded = files.upload(DIR)

TypeError: ignored

In [0]:
!ls -l

total 40
-rw-r--r-- 1 root root 31815 Sep 23 12:05 2.json
drwxr-xr-x 2 root root  4096 Sep 20 00:09 sample_data
drwxr-xr-x 2 root root  4096 Sep 23 11:38 themoviedb_people


In [0]:
#Import from local
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving 2.json to 2 (1).json
Saving 3.json to 3.json
Saving 5.json to 5.json
Saving 6.json to 6.json
Saving 8.json to 8.json
Saving 9.json to 9.json
Saving 11.json to 11.json
Saving 12.json to 12.json
Saving 13.json to 13.json
Saving 14.json to 14.json
Saving 15.json to 15.json
User uploaded file "2.json" with length 31815 bytes
User uploaded file "3.json" with length 31935 bytes
User uploaded file "5.json" with length 49141 bytes
User uploaded file "6.json" with length 34159 bytes
User uploaded file "8.json" with length 10552 bytes
User uploaded file "9.json" with length 15874 bytes
User uploaded file "11.json" with length 88847 bytes
User uploaded file "12.json" with length 110179 bytes
User uploaded file "13.json" with length 70864 bytes
User uploaded file "14.json" with length 60560 bytes
User uploaded file "15.json" with length 71419 bytes


IMPORTACIÓN DESDE GOOGLE DRIVE

In [0]:
#Esto sólo funciona en Google Colab porque tiene una librería de Google Colab.
#Más documentación en:
#https://colab.research.google.com/notebooks/io.ipynb
#en el apartado PyDrive.
#PyDrive documentación:
#https://gsuitedevs.github.io/PyDrive/docs/build/html/index.html

# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz

In [0]:
#!bunzip2 -k trial.tar.bz2 && tar -xvf trial.tar
#!ls -hl

trial/
trial/11.json
trial/12.json
trial/13.json
trial/14.json
trial/15.json
trial/2.json
trial/3.json
trial/5.json
trial/6.json
trial/8.json
trial/9.json
total 1.5G
-rw-r--r-- 1 root root 2.5K Sep 23 14:25 adc.json
-rw-r--r-- 1 root root 1.5G Sep 23 13:43 movies.zip
drwxr-xr-x 2 root root 4.0K Sep 20 00:09 sample_data
drwxr-xr-x 3 root root 4.0K Sep 23 13:46 themoviedb_data
drwxrwxrwx 2 1000 1000 4.0K Sep 23 11:59 trial
drwxr-xr-x 3 root root 4.0K Sep 23 13:06 trial_files
-rw-r--r-- 1 root root 580K Sep 23 14:25 trial.tar
-rw-r--r-- 1 root root 126K Sep 23 14:25 trial.tar.bz2
-rw-r--r-- 1 root root 176K Sep 23 13:03 trial.zip


In [0]:
#Estos ficheros están en mi google drive.

#Vínculo a la carpeta comprimida con todas las pelis en mi Google Drive:
#https://drive.google.com/open?id=1-jrX_p1VfgMceGIj7SK_eIef8syIGsdL

#movies
file_id = '1-jrX_p1VfgMceGIj7SK_eIef8syIGsdL'

downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile("movies.zip")

#trial
#file_id = '1ivGsdMnbPyHuxXPoZAnQVnjKp82-hq9o'

#downloaded = drive.CreateFile({'id': file_id})
#downloaded.GetContentFile("trial.zip")

In [4]:
!ls -lh

total 1.5G
-rw-r--r-- 1 root root 2.5K Oct 21 18:41 adc.json
-rw-r--r-- 1 root root 1.5G Oct 21 18:42 movies.zip
drwxr-xr-x 2 root root 4.0K Oct 18 16:40 sample_data


In [5]:
#!mkdir themoviedb_data
#!cd themoviedb_data && unzip -q ../trial.zip
#!ls -hl themoviedb_data

!mkdir themoviedb_data
!cd themoviedb_data && unzip -q ../movies.zip
!ls -hl themoviedb_data

warning [../movies.zip]:  76 extra bytes at beginning or within zipfile
  (attempting to process anyway)
error [../movies.zip]:  reported length of central directory is
  -76 bytes too long (Atari STZip zipfile?  J.H.Holm ZIPSPLIT 1.1
  zipfile?).  Compensating...
error:  expected central file header signature not found (file #409792).
  (please check that you have transferred or created the zipfile in the
  appropriate BINARY mode and that you have compiled UnZip properly)
total 12M
drwxr-xr-x 2 root root 12M Oct 21 18:43 movies


In [6]:
!ls -hl ./themoviedb_data/movies | head -20

total 4.3G
-rw-r--r-- 1 root root 7.4K Aug 26 18:10 100009.json
-rw-r--r-- 1 root root  22K Aug 25 12:44 10000.json
-rw-r--r-- 1 root root  24K Aug 26 18:10 100010.json
-rw-r--r-- 1 root root  18K Aug 26 18:10 100011.json
-rw-r--r-- 1 root root  16K Aug 26 18:10 100013.json
-rw-r--r-- 1 root root 6.8K Aug 26 18:10 100016.json
-rw-r--r-- 1 root root  19K Aug 26 18:10 100017.json
-rw-r--r-- 1 root root  32K Aug 25 12:44 10001.json
-rw-r--r-- 1 root root  16K Aug 26 18:10 100022.json
-rw-r--r-- 1 root root  17K Aug 26 18:10 100024.json
-rw-r--r-- 1 root root  20K Aug 26 18:10 100025.json
-rw-r--r-- 1 root root 3.1K Aug 26 18:10 100027.json
-rw-r--r-- 1 root root 5.7K Aug 26 18:10 100028.json
-rw-r--r-- 1 root root  32K Aug 25 12:44 10002.json
-rw-r--r-- 1 root root 7.5K Aug 26 18:10 100032.json
-rw-r--r-- 1 root root  19K Aug 26 18:10 100033.json
-rw-r--r-- 1 root root 3.3K Aug 26 18:10 100034.json
-rw-r--r-- 1 root root 6.2K Aug 26 18:10 100035.json
-rw-r--r-- 1 root root 2.4K Aug 26 18:

In [7]:
!ls -1 themoviedb_data/movies | wc -l

409791


In [0]:
#import ujson

#j3 = ujson.load(open(r"./themoviedb_data/trial/2.json"))
j3 = ujson.load(open(r"./themoviedb_data/movies/2.json"))

In [51]:
j3.keys()

dict_keys(['adult', 'backdrop_path', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count', 'alternative_titles', 'credits', 'external_ids', 'images', 'keywords', 'release_dates', 'recommendations', 'similar_movies', 'reviews'])

In [52]:
len(j3.keys())

34

In [53]:
j3['external_ids']

{'facebook_id': None,
 'imdb_id': 'tt0094675',
 'instagram_id': None,
 'twitter_id': None}

In [0]:
j3['external_ids']['facebook_id']

In [56]:
j3['external_ids']['imdb_id']

'tt0094675'

In [57]:
j3['alternative_titles']

{'titles': []}

In [13]:
list(j3.keys())

['adult',
 'backdrop_path',
 'belongs_to_collection',
 'budget',
 'genres',
 'homepage',
 'id',
 'imdb_id',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'video',
 'vote_average',
 'vote_count',
 'alternative_titles',
 'credits',
 'external_ids',
 'images',
 'keywords',
 'release_dates',
 'recommendations',
 'similar_movies',
 'reviews']

In [14]:
list(map(type, j3.values()))

[bool,
 str,
 NoneType,
 int,
 list,
 NoneType,
 int,
 str,
 str,
 str,
 str,
 float,
 str,
 list,
 list,
 str,
 int,
 int,
 list,
 str,
 str,
 str,
 bool,
 float,
 int,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict,
 dict]

In [15]:
list(zip(list(j3.keys()),list(map(type, j3.values()))))

[('adult', bool),
 ('backdrop_path', str),
 ('belongs_to_collection', NoneType),
 ('budget', int),
 ('genres', list),
 ('homepage', NoneType),
 ('id', int),
 ('imdb_id', str),
 ('original_language', str),
 ('original_title', str),
 ('overview', str),
 ('popularity', float),
 ('poster_path', str),
 ('production_companies', list),
 ('production_countries', list),
 ('release_date', str),
 ('revenue', int),
 ('runtime', int),
 ('spoken_languages', list),
 ('status', str),
 ('tagline', str),
 ('title', str),
 ('video', bool),
 ('vote_average', float),
 ('vote_count', int),
 ('alternative_titles', dict),
 ('credits', dict),
 ('external_ids', dict),
 ('images', dict),
 ('keywords', dict),
 ('release_dates', dict),
 ('recommendations', dict),
 ('similar_movies', dict),
 ('reviews', dict)]

In [17]:
import pandas as pd

pd.DataFrame(list(zip(list(j3.keys()),list(map(type, j3.values())))))

,0,1
0,adult,<class 'bool'>
1,backdrop_path,<class 'str'>
2,belongs_to_collection,<class 'NoneType'>
3,budget,<class 'int'>
4,genres,<class 'list'>
5,homepage,<class 'NoneType'>
6,id,<class 'int'>
7,imdb_id,<class 'str'>
8,original_language,<class 'str'>
9,original_title,<class 'str'>


In [18]:
pd.DataFrame([(x, type(j3[x])) for x in j3.keys()])

,0,1
0,adult,<class 'bool'>
1,backdrop_path,<class 'str'>
2,belongs_to_collection,<class 'NoneType'>
3,budget,<class 'int'>
4,genres,<class 'list'>
5,homepage,<class 'NoneType'>
6,id,<class 'int'>
7,imdb_id,<class 'str'>
8,original_language,<class 'str'>
9,original_title,<class 'str'>


In [77]:
isinstance(j3, dict)

True

In [78]:
pd.DataFrame([(x, type(j3[x])) for x in j3.keys() if isinstance(j3[x], dict)])

,0,1
0,alternative_titles,<class 'dict'>
1,credits,<class 'dict'>
2,external_ids,<class 'dict'>
3,images,<class 'dict'>
4,keywords,<class 'dict'>
5,release_dates,<class 'dict'>
6,recommendations,<class 'dict'>
7,similar_movies,<class 'dict'>
8,reviews,<class 'dict'>


In [79]:
pd.DataFrame([(x, y, type(j3[x][y])) for x in j3.keys() if isinstance(j3[x], dict) for y in j3[x].keys()])

,0,1,2
0,alternative_titles,titles,<class 'list'>
1,credits,cast,<class 'list'>
2,credits,crew,<class 'list'>
3,external_ids,imdb_id,<class 'str'>
4,external_ids,facebook_id,<class 'NoneType'>
5,external_ids,instagram_id,<class 'NoneType'>
6,external_ids,twitter_id,<class 'NoneType'>
7,images,backdrops,<class 'list'>
8,images,posters,<class 'list'>
9,keywords,keywords,<class 'list'>


In [83]:
pd.DataFrame([(x, 'NotDict', type(j3[x])) for x in j3.keys() if not isinstance(j3[x], dict)])

,0,1,2
0,adult,NotDict,<class 'bool'>
1,backdrop_path,NotDict,<class 'str'>
2,belongs_to_collection,NotDict,<class 'NoneType'>
3,budget,NotDict,<class 'int'>
4,genres,NotDict,<class 'list'>
5,homepage,NotDict,<class 'NoneType'>
6,id,NotDict,<class 'int'>
7,imdb_id,NotDict,<class 'str'>
8,original_language,NotDict,<class 'str'>
9,original_title,NotDict,<class 'str'>


In [0]:
dfCheck = pd.DataFrame([(x, 'NotDict', str(type(j3[x])), 'aux') for x in j3.keys() if not isinstance(j3[x], dict)], columns = ['key1', 'key2', 'type', 'aux'])
dfCheck = dfCheck.append(pd.DataFrame([(x, y, str(type(j3[x][y])), 'aux') for x in j3.keys() if isinstance(j3[x], dict) for y in j3[x].keys()], 
                                      columns = ['key1', 'key2', 'type', 'aux']))

In [119]:
dfCheck

,key1,key2,type,aux
0,adult,NotDict,<class 'bool'>,aux
1,backdrop_path,NotDict,<class 'str'>,aux
2,belongs_to_collection,NotDict,<class 'NoneType'>,aux
3,budget,NotDict,<class 'int'>,aux
4,genres,NotDict,<class 'list'>,aux
5,homepage,NotDict,<class 'NoneType'>,aux
6,id,NotDict,<class 'int'>,aux
7,imdb_id,NotDict,<class 'str'>,aux
8,original_language,NotDict,<class 'str'>,aux
9,original_title,NotDict,<class 'str'>,aux


In [121]:
dfCheck.groupby(['key1', 'key2', 'type']).count()

aux
key1                  key2          type                   
adult                 NotDict       <class 'bool'>        1
alternative_titles    titles        <class 'list'>        1
backdrop_path         NotDict       <class 'str'>         1
belongs_to_collection NotDict       <class 'NoneType'>    1
budget                NotDict       <class 'int'>         1
credits               cast          <class 'list'>        1
                      crew          <class 'list'>        1
external_ids          facebook_id   <class 'NoneType'>    1
                      imdb_id       <class 'str'>         1
                      instagram_id  <class 'NoneType'>    1
                      twitter_id    <class 'NoneType'>    1
genres                NotDict       <class 'list'>        1
homepage              NotDict       <class 'NoneType'>    1
id                    NotDict       <class 'int'>         1
images                backdrops     <class 'list'>        1
                      posters       <class 'list'>        1
imdb_id               NotDict       <class 'str'>         1
keywords              keywords      <class 'list'>        1
original_language     NotDict       <class 'str'>         1
original_title        NotDict       <class 'str'>         1
overview              NotDict       <class 'str'>         1
popularity            NotDict       <class 'float'>       1
poster_path           NotDict       <class 'str'>         1
production_companies  NotDict       <class 'list'>        1
production_countries  NotDict       <class 'list'>        1
recommendations       page          <class 'int'>         1
                      results       <class 'list'>        1
                      total_pages   <class 'int'>         1
                      total_results <class 'int'>         1
release_date          NotDict       <class 'str'>         1
release_dates         results       <class 'list'>        1
revenue               NotDict       <class 'int'>         1
reviews               page          <class 'int'>         1
                      results       <class 'list'>        1
                      total_pages   <class 'int'>         1
                      total_results <class 'int'>         1
runtime               NotDict       <class 'int'>         1
similar_movies        page          <class 'int'>         1
                      results       <class 'list'>        1
                      total_pages   <class 'int'>         1
                      total_results <class 'int'>         1
spoken_languages      NotDict       <class 'list'>        1
status                NotDict       <class 'str'>         1
tagline               NotDict       <class 'str'>         1
title                 NotDict       <class 'str'>         1
video                 NotDict       <class 'bool'>        1
vote_average          NotDict       <class 'float'>       1
vote_count            NotDict       <class 'int'>         1

In [35]:
j3['original_title']

'Ariel'

In [14]:
j3['overview']

"Taisto Kasurinen is a Finnish coal miner whose father has just committed suicide and who is framed for a crime he did not commit. In jail, he starts to dream about leaving the country and starting a new life. He escapes from prison but things don't go as planned..."

In [15]:
j3['popularity']

4.323

In [44]:
type(j3['popularity'])

float

In [45]:
j3['alternative_titles']

{'titles': []}

In [43]:
type(j3['alternative_titles'])

dict

In [46]:
type(j3['alternative_titles']['titles'])

list

In [0]:
import os
import pandas as pd
import timeit
import ujson
#Usamos ujson que parece que es el que hace la importación más rápida. Añadir la documentación donde se comenta.
import numpy as np #lo uso más adelante

path = "./themoviedb_data/movies"

In [9]:
tic=timeit.default_timer()

dfTypeCheck = pd.DataFrame()
dfAux = pd.DataFrame()

for root, dirs, files in os.walk(path):
    for f in files:
        if f.endswith('.json'):
            fp = os.path.join(root,f)
            with open(fp) as o:
                data = ujson.load(o)
                dfAux = pd.DataFrame([(x, 'NotDict', str(type(data[x])), 'aux') for x in data.keys() if not isinstance(data[x], dict)], 
                                                              columns = ['key1', 'key2', 'type', 'aux'])
                dfAux = dfAux.append(pd.DataFrame([(x, y, str(type(data[x][y])), 'aux') for x in data.keys() if isinstance(data[x], dict) 
                                                   for y in data[x].keys()], columns = ['key1', 'key2', 'type', 'aux']))
                dfTypeCheck = dfTypeCheck.append(dfAux.groupby(['key1','key2','type']).count())
                dfTypeCheck = dfTypeCheck.groupby(['key1','key2','type']).sum()
                
toc=timeit.default_timer()
toc - tic #elapsed time in seconds

22.87683205000002

In [0]:
#Hago un cambio en el siguiente bucle para ver si es más rápido.
#Después de comprobar que sí, le meto un contador para testear el tiempo de ejecución ya con todas las movies.

In [34]:
tic=timeit.default_timer()

dfTypeCheck = pd.DataFrame()
dfAux = pd.DataFrame()
intI = 0

for root, dirs, files in os.walk(path):
    for f in files:
        intI += 1
        if intI <= 500000:
            #Si el valor aquí es un múltiplo de 1000, la celda me devolverá un error. Porque dfAux quedará en blanco para la instrucción tras el "for".
            #Está finalmente resuelto.
            if f.endswith('.json'):
                fp = os.path.join(root,f)
                with open(fp) as o:
                
                    data = ujson.load(o)
                    dfAux = dfAux.append(pd.DataFrame([(x, 'NotDict', str(type(data[x])), 'aux') for x in data.keys() if not isinstance(data[x], dict)], 
                                                              columns = ['key1', 'key2', 'type', 'aux']))
                    dfAux = dfAux.append(pd.DataFrame([(x, y, str(type(data[x][y])), 'aux') for x in data.keys() if isinstance(data[x], dict) 
                                                   for y in data[x].keys()], columns = ['key1', 'key2', 'type', 'aux']))
                    if intI % 1000 == 0:
                        dfTypeCheck = dfTypeCheck.append(dfAux.groupby(['key1','key2','type']).count())
                        dfAux = pd.DataFrame()
                        print(intI)
        else:
            break
            
if intI % 1000 != 0:
    dfTypeCheck = dfTypeCheck.append(dfAux.groupby(['key1','key2','type']).count())
    dfAux = pd.DataFrame()

dfTypeCheck = dfTypeCheck.groupby(['key1','key2','type']).sum()

toc=timeit.default_timer()
toc - tic #elapsed time in seconds

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

1938.406806945

In [35]:
intI

409791

In [36]:
dfTypeCheck.head()

aux
key1                  key2    type                      
adult                 NotDict <class 'bool'>      409791
alternative_titles    titles  <class 'list'>      409791
backdrop_path         NotDict <class 'NoneType'>  270769
                              <class 'str'>       139022
belongs_to_collection NotDict <class 'NoneType'>  398774

In [37]:
dfTypeCheck.tail()

,,,aux
key1,key2,type,
tagline,NotDict,<class 'str'>,409791
title,NotDict,<class 'str'>,409791
video,NotDict,<class 'bool'>,409791
vote_average,NotDict,<class 'float'>,409791
vote_count,NotDict,<class 'int'>,409791


In [38]:
dfTypeCheck

aux
key1                  key2          type                      
adult                 NotDict       <class 'bool'>      409791
alternative_titles    titles        <class 'list'>      409791
backdrop_path         NotDict       <class 'NoneType'>  270769
                                    <class 'str'>       139022
belongs_to_collection NotDict       <class 'NoneType'>  398774
                      backdrop_path <class 'NoneType'>    4364
                                    <class 'str'>         6653
                      id            <class 'int'>        11017
                      name          <class 'str'>        11017
                      poster_path   <class 'NoneType'>    2647
                                    <class 'str'>         8370
budget                NotDict       <class 'int'>       409791
credits               cast          <class 'list'>      409791
                      crew          <class 'list'>      409791
external_ids          facebook_id   <class 'NoneType'>  406517
                                    <class 'str'>         3274
                      imdb_id       <class 'NoneType'>   61376
                                    <class 'str'>       348415
                      instagram_id  <class 'NoneType'>  408572
                                    <class 'str'>         1219
                      twitter_id    <class 'NoneType'>  408145
                                    <class 'str'>         1646
genres                NotDict       <class 'list'>      409791
homepage              NotDict       <class 'NoneType'>  365709
                                    <class 'str'>        44082
id                    NotDict       <class 'int'>       409791
images                backdrops     <class 'list'>      409791
                      posters       <class 'list'>      409791
imdb_id               NotDict       <class 'NoneType'>   61376
                                    <class 'str'>       348415
...                                                        ...
overview              NotDict       <class 'str'>       409791
popularity            NotDict       <class 'float'>     409791
poster_path           NotDict       <class 'NoneType'>   84776
                                    <class 'str'>       325015
production_companies  NotDict       <class 'list'>      409791
production_countries  NotDict       <class 'list'>      409791
recommendations       page          <class 'int'>       409791
                      results       <class 'list'>      409791
                      total_pages   <class 'int'>       409791
                      total_results <class 'int'>       409791
release_date          NotDict       <class 'str'>       409791
release_dates         results       <class 'list'>      409791
revenue               NotDict       <class 'int'>       409791
reviews               page          <class 'int'>       409791
                      results       <class 'list'>      409791
                      total_pages   <class 'int'>       409791
                      total_results <class 'int'>       409791
runtime               NotDict       <class 'NoneType'>  120276
                                    <class 'int'>       289515
similar_movies        page          <class 'int'>       409791
                      results       <class 'list'>      409791
                      total_pages   <class 'int'>       409791
                      total_results <class 'int'>       409791
spoken_languages      NotDict       <class 'list'>      409791
status                NotDict       <class 'str'>       409791
tagline               NotDict       <class 'str'>       409791
title                 NotDict       <class 'str'>       409791
video                 NotDict       <class 'bool'>      409791
vote_average          NotDict       <class 'float'>     409791
vote_count            NotDict       <class 'int'>       409791

[63 rows x 1 columns]

In [39]:
dfTypeCheck.shape

(63, 1)

In [40]:
#Esto me va a permitir tener acceso a las carpetas de mi drive desde Colab:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
!ls "/content/drive/My Drive/" -lh

total 3.2G
drwx------ 2 root root 4.0K Jul 14 07:08  20180714_Deep_Learning
drwx------ 2 root root 4.0K Jun 26  2016 'avast! Files (XT1068, UWbYzWPVmAmwlrlD)'
drwx------ 2 root root 4.0K Jul 18 18:38 'Colab Notebooks'
-rw------- 1 root root  44K Feb  4  2018  contacts.vcf
-rw------- 1 root root    1 Apr  9  2018  facturaIVA.gsheet
-rw------- 1 root root    1 Oct 11 07:38 'Mis cuentas.gsheet'
-rw------- 1 root root 1.5G Oct 12 15:09  movies.zip
-rw------- 1 root root    1 Oct 14 08:57 'Nuestras Cuentas.gsheet'
-rw------- 1 root root 1.8G Sep 24 21:07  people.zip
-rw------- 1 root root    1 Jul 24 18:17 'Propuesta de una idea.gslides'
drwx------ 2 root root 4.0K Apr  7  2015  Screencastify
-rw------- 1 root root    1 Sep 30  2017  Series.gdoc
-rw------- 1 root root    1 Jan  2  2016 'Trabajos Iñigo.gsheet'
-rw------- 1 root root  31M Sep 25 21:00  trial.zip
-rw------- 1 root root 2.7K Oct 14 11:04  typesCheck.csv
-rw------- 1 root root 6.6K Oct 14 11:04  typesCheck.xlsx
-rw------- 1 roo

In [0]:
dfTypeCheck.to_csv('typesCheck.csv')

In [47]:
!pip install openpyxl
import openpyxl

dfTypeCheck.to_excel('typesCheck.xlsx')

    100% |████████████████████████████████| 2.0MB 10.5MB/s 
  Running setup.py bdist_wheel for openpyxl ... - \ | done
  Stored in directory: /root/.cache/pip/wheels/3f/37/28/5ab3dffb7ff261e6fa21455ec9d157f95958e818c6b89f024c
  Running setup.py bdist_wheel for et-xmlfile ... - done
  Stored in directory: /root/.cache/pip/wheels/2a/77/35/0da0965a057698121fc7d8c5a7a9955cdbfb3cc4e2423cad39
Successfully built openpyxl et-xmlfile


In [0]:
#Copio los htmls de Colab a mi drive. Y después, desde Drive, ya los puedo bajar a local.
!cp typesCheck.* /content/drive/"My Drive"

In [49]:
!ls "/content/drive/My Drive/" -lh

total 3.2G
drwx------ 2 root root 4.0K Jul 14 07:08  20180714_Deep_Learning
drwx------ 2 root root 4.0K Jun 26  2016 'avast! Files (XT1068, UWbYzWPVmAmwlrlD)'
drwx------ 2 root root 4.0K Jul 18 18:38 'Colab Notebooks'
-rw------- 1 root root  44K Feb  4  2018  contacts.vcf
-rw------- 1 root root    1 Apr  9  2018  facturaIVA.gsheet
-rw------- 1 root root    1 Oct 11 07:38 'Mis cuentas.gsheet'
-rw------- 1 root root 1.5G Oct 12 15:09  movies.zip
-rw------- 1 root root    1 Oct 14 08:57 'Nuestras Cuentas.gsheet'
-rw------- 1 root root 1.8G Sep 24 21:07  people.zip
-rw------- 1 root root    1 Jul 24 18:17 'Propuesta de una idea.gslides'
drwx------ 2 root root 4.0K Apr  7  2015  Screencastify
-rw------- 1 root root    1 Sep 30  2017  Series.gdoc
-rw------- 1 root root    1 Jan  2  2016 'Trabajos Iñigo.gsheet'
-rw------- 1 root root  31M Sep 25 21:00  trial.zip
-rw------- 1 root root 2.7K Oct 14 11:04  typesCheck.csv
-rw------- 1 root root 6.6K Oct 14 11:04  typesCheck.xlsx
-rw------- 1 roo

In [0]:
#No vale de nada cambiar el orden!!! porque un diccionario no tiene orden. Lo hice aquí, aunque no sirvió de nada.
#Mantengo la celda por si me interesa ver este orden.

def data_iterator(path):
    intI = 0
    for root, dirs, files in os.walk(path):
        for f in files:
            intI += 1
            if intI <= 1000:
                if f.endswith('.json'):
                    fp = os.path.join(root,f)
                    with open(fp) as o:
                        data = ujson.load(o)
                
                    yield {"id": data["id"], "title": data["title"], "adult": data["adult"], "genres": data["genres"], 
                           "credits_cast": data["credits"]["cast"], "credits_crew": data["credits"]["crew"],
                           "original_title": data["original_title"], "overview": data["overview"],
                           "release_date": data["release_date"], "release_dates": data["release_dates"]["results"],
                           "keywords": data["keywords"]["keywords"], "original_language": data["original_language"],
                           "spoken_languages": data["spoken_languages"], "status": data["status"],
                           "titles": data["alternative_titles"]["titles"], 
                           "production_companies": data["production_companies"], "production_countries": data["production_countries"],
                           "budget": data["budget"], "revenue": data["revenue"], "popularity": data["popularity"], "runtime": data["runtime"],
                           "belongs_to_collection": data["belongs_to_collection"],
                           "external_ids_facebook_id": data["external_ids"]["facebook_id"],
                           "external_ids_imdb_id": data["external_ids"]["imdb_id"],
                           "external_ids_instagram_id": data["external_ids"]["instagram_id"],
                           "external_ids_twitter_id": data["external_ids"]["twitter_id"],
                           "homepage": data["homepage"], 
                           "images_backdrops": data["images"]["backdrops"], "images_posters": data["images"]["posters"],
                           "poster_path": data["poster_path"], "backdrop_path": data["backdrop_path"],
                           "recommendations_page": data["recommendations"]["page"],
                           "recommendations_results": data["recommendations"]["results"],
                           "recommendations_total_pages": data["recommendations"]["total_pages"],
                           "recommendations_total_results": data["recommendations"]["total_results"],
                           "reviews_page": data["reviews"]["page"],
                           "reviews_results": data["reviews"]["results"],
                           "reviews_total_pages": data["reviews"]["total_pages"],
                           "reviews_total_results": data["reviews"]["total_results"],
                           "similar_movies_page": data["similar_movies"]["page"],
                           "similar_movies_results": data["similar_movies"]["results"],
                           "similar_movies_total_pages": data["similar_movies"]["total_pages"],
                           "similar_movies_total_results": data["similar_movies"]["total_results"],
                           "tagline": data["tagline"], "video": data["video"],
                           "vote_average": data["vote_average"], "vote_count": data["vote_count"]}

dfMovies = pd.DataFrame(data_iterator(path))
dfMovies.set_index('id', inplace=True)

In [0]:
#Lo importo todo (o lo intento)

#Usamos un generator, que es más eficiente en cuanto a memoria. "yield" va asociado al generator.
#Poner la nota: importo todo, aunque sé que lo voy a usar, por si alguien puede querer usarlo.
#Desanido el primer nivel de diccionarios, salvo belongs_to_collection (porque a veces es nulo).
#Me queda tratar las listas que contienen diccionarios. Y belongs_to_collection
#¿Qué pasa con UTF8? !!!!!!!!!!!!!!!!!!!!!!!!!!
#No vale de nada cambiar el orden!!! Así que mantengo el alfabético del csv

def data_iterator(path):
    intI = 0
    for root, dirs, files in os.walk(path):
        for f in files:
            intI += 1
            if intI <= 500000:
                if f.endswith('.json'):
                    fp = os.path.join(root,f)
                    with open(fp) as o:
                        data = ujson.load(o)
                
                    if intI % 10000 == 0:
                        print(intI)
                    
                    yield {"adult": data["adult"], 
                       "titles": data["alternative_titles"]["titles"], "backdrop_path": data["backdrop_path"],
                       "belongs_to_collection": data["belongs_to_collection"],
                       "budget": data["budget"], "credits_cast": data["credits"]["cast"], "credits_crew": data["credits"]["crew"],
                       "external_ids_facebook_id": data["external_ids"]["facebook_id"],
                       "external_ids_imdb_id": data["external_ids"]["imdb_id"],
                       "external_ids_instagram_id": data["external_ids"]["instagram_id"],
                       "external_ids_twitter_id": data["external_ids"]["twitter_id"],
                       "genres": data["genres"], "homepage": data["homepage"], "id": data["id"],
                       "images_backdrops": data["images"]["backdrops"], "images_posters": data["images"]["posters"],
                       "keywords": data["keywords"]["keywords"], "original_language": data["original_language"],
                       "original_title": data["original_title"], "overview": data["overview"],
                       "popularity": data["popularity"], "poster_path": data["poster_path"],
                       "production_companies": data["production_companies"], "production_countries": data["production_countries"],
                       "recommendations_page": data["recommendations"]["page"],
                       "recommendations_results": data["recommendations"]["results"],
                       "recommendations_total_pages": data["recommendations"]["total_pages"],
                       "recommendations_total_results": data["recommendations"]["total_results"],
                       "release_date": data["release_date"], "release_dates": data["release_dates"]["results"],
                       "revenue": data["revenue"], 
                       "reviews_page": data["reviews"]["page"],
                       "reviews_results": data["reviews"]["results"],
                       "reviews_total_pages": data["reviews"]["total_pages"],
                       "reviews_total_results": data["reviews"]["total_results"],
                       "runtime": data["runtime"],
                       "similar_movies_page": data["similar_movies"]["page"],
                       "similar_movies_results": data["similar_movies"]["results"],
                       "similar_movies_total_pages": data["similar_movies"]["total_pages"],
                       "similar_movies_total_results": data["similar_movies"]["total_results"],
                       "spoken_languages": data["spoken_languages"], "status": data["status"],
                       "tagline": data["tagline"], "title": data["title"], "video": data["video"],
                       "vote_average": data["vote_average"], "vote_count": data["vote_count"]}
                    
            else:
                break          

tic=timeit.default_timer()
dfMovies = pd.DataFrame(data_iterator(path))
dfMovies.set_index('id', inplace=True)
toc=timeit.default_timer()
toc - tic #elapsed time in seconds


10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000


In [8]:
#Lo importo todo. Tengo que reducir el número de campos para que me quepa en memoria.
#Excluyo: backdrop_path, external_ids, homepage, images_backdrops, images_posters, poster_path, similar_movies_page/results/total_pages/total_results,
#video, recommendations_page/results/total_pages/total_results
#Recommendations: películas recomendadas para una película. No es lo mismo que similar_movies.

#Usamos un generator, que es más eficiente en cuanto a memoria. "yield" va asociado al generator.
#Poner la nota: importo todo, aunque sé que lo voy a usar, por si alguien puede querer usarlo.
#Desanido el primer nivel de diccionarios, salvo belongs_to_collection (porque a veces es nulo).
#Me queda tratar las listas que contienen diccionarios. Y belongs_to_collection
#¿Qué pasa con UTF8? !!!!!!!!!!!!!!!!!!!!!!!!!!
#No vale de nada cambiar el orden!!! Así que mantengo el alfabético del csv

dfMovies = pd.DataFrame()

def data_iterator(path):
    intI = 0
    for root, dirs, files in os.walk(path):
        for f in files:
            intI += 1
            if intI <= 500000:
                if f.endswith('.json'):
                    fp = os.path.join(root,f)
                    with open(fp) as o:
                        data = ujson.load(o)
                
                    if intI % 10000 == 0:
                        print(intI)
                    
                    yield {"adult": data["adult"], 
                       "titles": data["alternative_titles"]["titles"], 
                       "belongs_to_collection": data["belongs_to_collection"],
                       "budget": data["budget"], "credits_cast": data["credits"]["cast"], "credits_crew": data["credits"]["crew"],
                       "genres": data["genres"], "id": data["id"],
                       "keywords": data["keywords"]["keywords"], "original_language": data["original_language"],
                       "original_title": data["original_title"], "overview": data["overview"],
                       "popularity": data["popularity"], 
                       "production_companies": data["production_companies"], "production_countries": data["production_countries"],
                       "release_date": data["release_date"], "release_dates": data["release_dates"]["results"],
                       "revenue": data["revenue"], 
                       "reviews_page": data["reviews"]["page"],
                       "reviews_results": data["reviews"]["results"],
                       "reviews_total_pages": data["reviews"]["total_pages"],
                       "reviews_total_results": data["reviews"]["total_results"],
                       "runtime": data["runtime"],
                       "spoken_languages": data["spoken_languages"], "status": data["status"],
                       "tagline": data["tagline"], "title": data["title"], 
                       "vote_average": data["vote_average"], "vote_count": data["vote_count"]}
                    
            else:
                break          

tic=timeit.default_timer()
dfMovies = pd.DataFrame(data_iterator(path))
dfMovies.set_index('id', inplace=True)
toc=timeit.default_timer()
toc - tic #elapsed time in seconds


10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000


310.336914132

In [9]:
dfMovies.head()

,adult,belongs_to_collection,budget,credits_cast,credits_crew,genres,keywords,original_language,original_title,overview,...,reviews_total_pages,reviews_total_results,runtime,spoken_languages,status,tagline,title,titles,vote_average,vote_count
id,,,,,,,,,,,,,,,,,,,,,
263453,False,None,0,"[{'cast_id': 1, 'character': 'Obergefreiter Bl...","[{'credit_id': '53429416c3a368151e003457', 'de...",[],[],de,Barras Heute,,...,0,0,NaN,[],Released,,Barras Heute,[],0.0,0
544487,False,None,0,[],[],[],[],en,HANTU KAK LIMAH,Kak Limah is discovered dead by villager. Sinc...,...,0,0,111.0,[],Released,MALAY,HANTU KAK LIMAH,[],4.0,1
199256,False,None,0,"[{'cast_id': 2, 'character': 'حكمدار العاصمة',...","[{'credit_id': '53a682fcc3a36874950021da', 'de...","[{'id': 18, 'name': 'Drama'}]",[],ar,حياة أو موت,A man suffers from a heart attack so he sends ...,...,0,0,95.0,"[{'iso_639_1': 'ar', 'name': 'العربية'}]",Released,,Life or Death,"[{'iso_3166_1': 'EG', 'title': 'Hayat ou maut'...",6.5,2
261242,False,"{'id': 486877, 'name': '天使三部曲', 'poster_path':...",0,"[{'cast_id': 1, 'character': 'Siu Hung', 'cred...","[{'credit_id': '541e8f050e0a26075400149c', 'de...","[{'id': 28, 'name': 'Action'}]","[{'id': 570, 'name': 'rape'}, {'id': 1419, 'na...",cn,終極獵殺,Chu Chi Kit returns to HK from Taiwan after ma...,...,0,0,89.0,"[{'iso_639_1': 'cn', 'name': '广州话 / 廣州話'}, {'i...",Released,,Hunting List,"[{'iso_3166_1': 'HK', 'title': 'Zhong ji lie s...",6.0,1
297919,True,None,0,"[{'cast_id': 1, 'character': '', 'credit_id': ...","[{'credit_id': '54415fb30e0a26579e000010', 'de...",[],[],en,The Wrong Girl,Maria is a girl with a lot of baggage. And Buc...,...,0,0,82.0,[],Released,,The Wrong Girl,[],0.0,0


In [10]:
dfMovies.tail()

,adult,belongs_to_collection,budget,credits_cast,credits_crew,genres,keywords,original_language,original_title,overview,...,reviews_total_pages,reviews_total_results,runtime,spoken_languages,status,tagline,title,titles,vote_average,vote_count
id,,,,,,,,,,,,,,,,,,,,,
236245,False,None,0,[],"[{'credit_id': '52fe4e57c3a36847f828af51', 'de...",[],[],en,The Tempermental Lion,A lion with a Bert Lahr voice can't stand bein...,...,0,0,NaN,[],Released,,The Tempermental Lion,[],0.0,0
272736,False,None,0,"[{'cast_id': 16, 'character': 'Antonella de Ca...","[{'credit_id': '53878363c3a3681aed003556', 'de...",[],[],it,Bologna due agosto: I giorni della collera,,...,0,0,NaN,"[{'iso_639_1': 'it', 'name': 'Italiano'}]",Released,,Bologna due agosto: I giorni della collera,[],5.0,2
151533,False,None,0,"[{'cast_id': 4, 'character': '', 'credit_id': ...","[{'credit_id': '5556f7729251411e4d005170', 'de...","[{'id': 18, 'name': 'Drama'}]",[],pt,Latitude Zero,Lena owns a forgotten bar next to a highway wh...,...,0,0,85.0,[],Released,,Latitude Zero,[],5.0,1
538894,False,None,0,[],"[{'credit_id': '5b5de352925141521a02db54', 'de...",[],[],de,Wartesaal der Weltgeschichte,Short by Herbert Viktor.,...,0,0,17.0,[],Released,,Wartesaal der Weltgeschichte,[],0.0,0
265291,False,None,0,"[{'cast_id': 1, 'character': '', 'credit_id': ...","[{'credit_id': '5463703bc3a368124d0022d5', 'de...","[{'id': 36, 'name': 'History'}, {'id': 99, 'na...",[],en,Vittorio D,A documentary about Vittorio de Sica with clip...,...,0,0,93.0,"[{'iso_639_1': 'it', 'name': 'Italiano'}, {'is...",Released,,Vittorio D,[],7.7,3


In [11]:
dfMovies.shape

(409791, 28)

In [12]:
dfMovies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'credits_cast',
       'credits_crew', 'genres', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'release_dates', 'revenue',
       'reviews_page', 'reviews_results', 'reviews_total_pages',
       'reviews_total_results', 'runtime', 'spoken_languages', 'status',
       'tagline', 'title', 'titles', 'vote_average', 'vote_count'],
      dtype='object')

In [45]:
dfMovies[['title','titles']]

,title,titles
id,,
259834,Wild Seed,"[{'iso_3166_1': 'US', 'title': 'Wild Seeds', '..."
141996,Quattro Noza,[]
178650,Gino Vannelli Live in Montréal,[]
332474,We Came to Sweat,"[{'iso_3166_1': 'US', 'title': 'We Came to Swe..."
519311,Panty Pervert 2,[]
369324,The Art of Recovery,[]
480129,The Deluge,[]
359264,I Think I'm In Love,[]
381385,Closer,[]


In [51]:
dfMovies[dfMovies.index==349565].values

array([[True, None, 0, list([]), list([]), list([]), list([]), 'ko',
        '헬로우 변강쇠', '', 0.001, list([]), list([]), 1, list([]), 1, 0,
        '1995-05-04',
        list([{'iso_3166_1': 'KR', 'release_dates': [{'certification': '19', 'iso_639_1': '', 'note': '', 'release_date': '1995-05-04T00:00:00.000Z', 'type': 3}]}]),
        0, 1, list([]), 0, 0, nan, list([]), 'Released', '', '헬로우 변강쇠',
        list([{'iso_3166_1': 'KR', 'title': '헬로우 변강쇠', 'type': ''}]),
        0.0, 0]], dtype=object)

In [0]:
#Excluyo: backdrop_path, external_ids, homepage, images_backdrops, images_posters, poster_path, similar_movies_page/results/total_pages/total_results,
#video, recommendations_page/results/total_pages/total_results
#Recommendations: películas recomendadas para una película. No es lo mismo que similar_movies.
dfMovies[dfMovies['backdrop_path'].notnull()]['backdrop_path'].head(10)

In [0]:
dfMovies[dfMovies['homepage'].notnull()]['homepage'].head(10)

In [0]:
dfMovies[dfMovies['poster_path'].notnull()]['poster_path'].head(10)

In [0]:
dfMovies[dfMovies['video'].notnull()]['video'].head(10)

In [48]:
dfMovies[dfMovies['recommendations_page'].notnull()][['recommendations_page', 'recommendations_results', 
                                                      'recommendations_total_pages', 'recommendations_total_results']].head(10)

,recommendations_page,recommendations_results,recommendations_total_pages,recommendations_total_results
id,,,,
259834,1,[],1,0
141996,1,[],1,0
178650,1,[],1,0
332474,1,[],1,0
519311,1,[],1,0
369324,1,[],1,0
480129,1,[],1,0
359264,1,[],1,0
381385,1,[],1,0


In [53]:
dfMovies[dfMovies.index==76341][['recommendations_page', 'recommendations_results', 
                                                      'recommendations_total_pages', 'recommendations_total_results']]

,recommendations_page,recommendations_results,recommendations_total_pages,recommendations_total_results
id,,,,
76341,1,"[{'adult': False, 'backdrop_path': '/9X3cDZb4G...",2,40


In [0]:
dfMovies[dfMovies.index==76341]['recommendations_results'].tolist()

In [68]:
type(dfMovies[dfMovies.index==76341]['recommendations_results'])

pandas.core.series.Series

In [21]:
dfMovies[dfMovies.index==182224].values

array([[False, None, 0,
        list([{'cast_id': 1, 'character': 'Rapaz', 'credit_id': '52fe4c7e9251416c7511c1df', 'gender': 0, 'id': 1165009, 'name': 'Fábio Santos', 'order': 0, 'profile_path': None}, {'cast_id': 2, 'character': 'Rapariga', 'credit_id': '52fe4c7e9251416c7511c1e3', 'gender': 0, 'id': 1165010, 'name': 'Cristina Gomes', 'order': 1, 'profile_path': None}]),
        list([{'credit_id': '52fe4c7e9251416c7511c1e9', 'department': 'Directing', 'gender': 0, 'id': 1160514, 'job': 'Director', 'name': 'Eduardo Rodrigues', 'profile_path': None}, {'credit_id': '52fe4c7e9251416c7511c1ef', 'department': 'Writing', 'gender': 0, 'id': 1160516, 'job': 'Story', 'name': 'Tony Simões', 'profile_path': None}, {'credit_id': '52fe4c7e9251416c7511c1f5', 'department': 'Editing', 'gender': 0, 'id': 1160514, 'job': 'Editor', 'name': 'Eduardo Rodrigues', 'profile_path': None}, {'credit_id': '52fe4c7e9251416c7511c1fb', 'department': 'Writing', 'gender': 0, 'id': 1160514, 'job': 'Writer', 'name': 'E

In [15]:
dfMovies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'credits_cast',
       'credits_crew', 'genres', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'release_dates', 'revenue',
       'reviews_page', 'reviews_results', 'reviews_total_pages',
       'reviews_total_results', 'runtime', 'spoken_languages', 'status',
       'tagline', 'title', 'titles', 'vote_average', 'vote_count'],
      dtype='object')

In [33]:
dfMovies[dfMovies['belongs_to_collection'].notnull()]['belongs_to_collection'].head(10)

id
115873    {'id': 427397, 'name': 'The Angrez Collection'...
321528    {'id': 334996, 'name': 'Son of Batman Collecti...
170610    {'id': 170618, 'name': 'Nicky Cruz Collection'...
345882    {'id': 345865, 'name': 'Emmanuelle 2000 Collec...
133879    {'id': 24717, 'name': 'Johan Falk GSI Collecti...
13981     {'id': 87800, 'name': 'Appleseed Collection', ...
12712     {'id': 11118, 'name': 'The Olsen Gang Collecti...
33534     {'id': 122123, 'name': 'Three Heroes Collectio...
47631     {'id': 47629, 'name': 'Mr. Boogedy Collection'...
494480    {'id': 495185, 'name': 'Aadu Collection', 'pos...
Name: belongs_to_collection, dtype: object

In [41]:
dfMovies[dfMovies.index==115873]['belongs_to_collection'].map(type)

id
115873    <class 'dict'>
Name: belongs_to_collection, dtype: object

In [43]:
dfMovies[dfMovies.index==115873]['belongs_to_collection'].tolist()

[{'backdrop_path': '/wj8MFAYnKz0k2LcERbpnxhL5cIK.jpg',
  'id': 427397,
  'name': 'The Angrez Collection',
  'poster_path': '/aGTwpTqBiqRmL3TYVm7d0lsJFJG.jpg'}]

In [0]:
#Me bastaría con quedarme con el id de la collección y su nombre (como mucho!)

In [9]:
#Lo importo todo. Tengo que reducir el número de campos para que me quepa en memoria.
#Excluyo: backdrop_path, external_ids, homepage, images_backdrops, images_posters, poster_path, similar_movies_page/results/total_pages/total_results,
#video, recommendations_page/results/total_pages/total_results
#Recommendations: películas recomendadas para una película. No es lo mismo que similar_movies.
#Para hacer esto debería hacer una importación inicial de todos los campos y luego, una vez decidido, importo todo.


#Usamos un generator, que es más eficiente en cuanto a memoria. "yield" va asociado al generator.
#Poner la nota: importo todo, aunque sé que lo voy a usar, por si alguien puede querer usarlo.
#Desanido el primer nivel de diccionarios. Para belongs_to_collection, que a veces es nulo en lugar de un diccionario, tomo dos keys.
#Me queda tratar las listas que contienen diccionarios. 
#¿Qué pasa con UTF8? !!!!!!!!!!!!!!!!!!!!!!!!!!
#No vale de nada cambiar el orden!!! Así que mantengo el alfabético del csv
#Por mi código anterior ya sé que adult es un booleano y que no tiene nulos. Así que puedo filtrarlo por adult = False

dfMovies = pd.DataFrame()

def data_iterator(path):
    intI = 0
    for root, dirs, files in os.walk(path):
        for f in files:
            intI += 1
            if intI <= 500000:
                if f.endswith('.json'):
                    fp = os.path.join(root,f)
                    with open(fp) as o:
                        data = ujson.load(o)
                
                    if intI % 10000 == 0:
                        print(intI)
                    
                    result = {"titles": data["alternative_titles"]["titles"], 
                       "budget": data["budget"], "credits_cast": data["credits"]["cast"], "credits_crew": data["credits"]["crew"],
                       "genres": data["genres"], "id": data["id"],
                       "keywords": data["keywords"]["keywords"], "original_language": data["original_language"],
                       "original_title": data["original_title"], "overview": data["overview"],
                       "popularity": data["popularity"], 
                       "production_companies": data["production_companies"], "production_countries": data["production_countries"],
                       "release_date": data["release_date"], "release_dates": data["release_dates"]["results"],
                       "revenue": data["revenue"], 
                       "reviews_page": data["reviews"]["page"],
                       "reviews_results": data["reviews"]["results"],
                       "reviews_total_pages": data["reviews"]["total_pages"],
                       "reviews_total_results": data["reviews"]["total_results"],
                       "runtime": data["runtime"],
                       "spoken_languages": data["spoken_languages"], "status": data["status"],
                       "tagline": data["tagline"], "title": data["title"], 
                       "vote_average": data["vote_average"], "vote_count": data["vote_count"]}
                    
                    if data["belongs_to_collection"] is None:
                        result["belongs_to_collection_id"] = None
                        result["belongs_to_collection_name"] = None
                    else:
                        result["belongs_to_collection_id"] = data["belongs_to_collection"]["id"]
                        result["belongs_to_collection_name"] = data["belongs_to_collection"]["name"]
                    
                    if data['adult'] == False:
                        yield result
            else:
                break          

tic=timeit.default_timer()
dfMovies = pd.DataFrame(data_iterator(path))
dfMovies.set_index('id', inplace=True)
toc=timeit.default_timer()
toc - tic #elapsed time in seconds

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000


292.06326689799994

In [26]:
dfMovies.shape

(393239, 28)

In [21]:
dfMovies['title'].count()

409791

In [23]:
dfMovies['adult'].count()

409791

In [24]:
dfMovies.groupby('adult')['title'].count()

adult
False    393239
True      16552
Name: title, dtype: int64

In [30]:
dfTest[dfTest['belongs_to_collection_id'].notnull()][['belongs_to_collection_id','belongs_to_collection_name']].head(5)

,belongs_to_collection_id,belongs_to_collection_name
id,,
134232,392686.0,The Three Mesquiteers
10035,404111.0,100 Girls Collection
372988,372993.0,Vanishing Son Collection
81204,324384.0,Příhody malého lovce z doby kamenné
257118,147765.0,The Dog Who Saved Christmas Collection


In [31]:
dfTest[dfTest['belongs_to_collection_id'].isnull()][['belongs_to_collection_id','belongs_to_collection_name']].head(5)

,belongs_to_collection_id,belongs_to_collection_name
id,,
2148,NaN,None
144552,NaN,None
449513,NaN,None
107116,NaN,None
537431,NaN,None


In [0]:
#TITLES *****************************************

In [33]:
dfTest['titles'].head(10)

id
2148      [{'iso_3166_1': 'IT', 'title': 'Cotton Club', ...
144552                                                   []
449513                                                   []
107116                                                   []
537431                                                   []
10000     [{'iso_3166_1': 'US', 'title': 'The Snail's St...
360079                                                   []
336511                                                   []
73339                                                    []
515066                                                   []
Name: titles, dtype: object

In [31]:
np.sort(dfMovies['titles'].map(len).unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 38, 40, 42, 44, 45, 48, 51, 56])

In [36]:
#Es una variable muy poco informada
dfMovies['titles'].map(len).value_counts()

0     303051
1      50111
2      17572
3       8246
4       4510
5       2703
6       1798
7       1230
8        872
9        643
10       537
11       426
12       332
13       221
14       208
15       136
16       120
17        91
18        73
19        63
20        42
21        36
22        36
23        33
25        28
24        22
26        17
27        15
28        14
34        10
29         7
30         7
31         6
35         5
33         4
38         2
45         2
42         2
51         2
40         1
36         1
44         1
32         1
48         1
56         1
Name: titles, dtype: int64

In [33]:
#No sé si me harán falta los títulos para tener una versión en inglés de las pelis chinas, etc.
#De momento, lo voy a dejar sin tratar, porque no creo que lo vaya a utilizar.
dfMovies[dfMovies['titles'].map(len)==33]['titles'].tolist()

[[{'iso_3166_1': 'TW', 'title': '變形金剛4：絕跡重生', 'type': ''},
  {'iso_3166_1': 'FR',
   'title': "Transformers : l'âge de l'extinction",
   'type': ''},
  {'iso_3166_1': 'AR', 'title': 'Transformers 4', 'type': ''},
  {'iso_3166_1': 'BR', 'title': 'Transformers: A Era da Extinção', 'type': ''},
  {'iso_3166_1': 'CL',
   'title': 'Transformers 4: La era de la extinción',
   'type': ''},
  {'iso_3166_1': 'ES',
   'title': 'Transformers 4: La era de la extinción',
   'type': ''},
  {'iso_3166_1': 'FI', 'title': 'Transformers - Tuhon aikakausi', 'type': ''},
  {'iso_3166_1': 'HU', 'title': 'Transformers: A kihalás kora', 'type': ''},
  {'iso_3166_1': 'LT', 'title': 'Transformeriai 4', 'type': ''},
  {'iso_3166_1': 'PT', 'title': 'Transformers: Era da Extinção', 'type': ''},
  {'iso_3166_1': 'RS', 'title': 'Transformersi - Doba izumiranja', 'type': ''},
  {'iso_3166_1': 'RU', 'title': 'Трансформеры: Эпоха истребления', 'type': ''},
  {'iso_3166_1': 'CN', 'title': '变形金刚4', 'type': ''},
  {'iso_

In [40]:
dfMovies[dfMovies['titles'].map(len)>0]['titles'].iloc[0]

[{'iso_3166_1': 'US',
  'title': "Wagner's Master Singer, Hitler's Siegfried - The Life and Times of Max Lorenz",
  'type': 'USA (long title)'},
 {'iso_3166_1': 'DE',
  'title': 'Wagners Meistersänger, Hitlers Siegfried - Auf den Spuren von Max Lorenz',
  'type': 'Germany (long title)'}]

In [43]:
dfMovies[dfMovies['titles'].map(len)>0]['titles'].iloc[0][0].keys()

dict_keys(['iso_3166_1', 'title', 'type'])

In [0]:
#CAST ****************************************

In [44]:
#import numpy as np
np.sort(dfMovies['credits_cast'].map(len).unique())

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 120, 121, 122, 123, 124, 125, 127, 128, 130, 131, 133,
       134, 135, 136, 137, 139, 140, 142, 143, 145, 146, 151, 152, 153,
       154, 158, 159, 164, 165, 168, 170, 171, 172, 183, 185, 194, 208,
       213, 224, 268, 313])

In [45]:
dfMovies['credits_cast'].map(len).value_counts()
#Tenemos 124820 pelis sin actores!!! Habrá que ver a qué corresponde. Y qué hacer con ellas.

0      124820
1       27846
4       25515
5       24594
6       21337
3       19676
2       18525
7       17688
8       16379
9       13341
10      12699
11       9759
12       8586
15       8218
13       6861
14       6048
16       4352
17       3375
18       2834
19       2394
20       2161
21       1818
22       1481
23       1338
24       1106
25        952
26        878
27        758
28        724
29        574
        ...  
171         2
136         2
122         2
153         1
158         1
268         1
159         1
172         1
165         1
168         1
170         1
185         1
194         1
154         1
116         1
152         1
151         1
145         1
142         1
140         1
139         1
134         1
133         1
130         1
127         1
124         1
121         1
313         1
213         1
224         1
Name: credits_cast, Length: 160, dtype: int64

In [46]:
#Un ejemplo:
dfMovies[dfMovies.index ==76341]['credits_cast'].map(len)

id
76341    60
Name: credits_cast, dtype: int64

In [45]:
dfMovies[dfMovies.index ==76341]['credits_cast'].iloc[0]

[{'cast_id': 2,
  'character': 'Max Rockatansky',
  'credit_id': '52fe4933c3a368484e11f773',
  'gender': 2,
  'id': 2524,
  'name': 'Tom Hardy',
  'order': 0,
  'profile_path': '/4CR1D9VLWZcmGgh4b6kKuY2NOel.jpg'},
 {'cast_id': 35,
  'character': 'Imperator Furiosa',
  'credit_id': '560e8a2bc3a368681f00b80c',
  'gender': 1,
  'id': 6885,
  'name': 'Charlize Theron',
  'order': 1,
  'profile_path': '/aHOB9UMgSI7MlXF3GMZaVQXqfQg.jpg'},
 {'cast_id': 4,
  'character': 'Nux',
  'credit_id': '52fe4933c3a368484e11f77b',
  'gender': 2,
  'id': 3292,
  'name': 'Nicholas Hoult',
  'order': 2,
  'profile_path': '/27NbnWKzco1CCDHtR2AVVab8Dli.jpg'},
 {'cast_id': 21,
  'character': 'Immortan Joe',
  'credit_id': '55549e64c3a3682086002b85',
  'gender': 2,
  'id': 26060,
  'name': 'Hugh Keays-Byrne',
  'order': 3,
  'profile_path': '/lhYUcPimMMy7Ltp58NkB3RuJzSv.jpg'},
 {'cast_id': 22,
  'character': 'Slit',
  'credit_id': '55549e84c3a36820800028e6',
  'gender': 2,
  'id': 1056053,
  'name': 'Josh Helma

In [40]:
type(dfMovies[dfMovies.index ==76341]['credits_cast'])

pandas.core.series.Series

In [47]:
type(dfMovies[dfMovies.index ==76341]['credits_cast'].iloc[0])

list

In [49]:
#Veo en la documentación que estas keys son iguales en todo el "Cast"
dfMovies[dfMovies.index ==76341]['credits_cast'].iloc[0][0].keys()

dict_keys(['cast_id', 'character', 'credit_id', 'gender', 'id', 'name', 'order', 'profile_path'])

In [48]:
dfMovies[dfMovies.index ==76341]['credits_cast'].iloc[0][0]

{'cast_id': 2,
 'character': 'Max Rockatansky',
 'credit_id': '52fe4933c3a368484e11f773',
 'gender': 2,
 'id': 2524,
 'name': 'Tom Hardy',
 'order': 0,
 'profile_path': '/4CR1D9VLWZcmGgh4b6kKuY2NOel.jpg'}

In [0]:
#Así que tengo un data series donde cada elemento es una lista de longitud variable de diccionarios todos con las mismas claves.
#De momento, cogeré todas salvo el profile_path, que no lo voy a usar con seguridad.

In [0]:
listTest =  dfMovies[dfMovies.index ==76341]['credits_cast'].iloc[0]

In [0]:
dfTest = pd.DataFrame()

def list_iterator(input_list):
    for dict in input_list:
        output = {'cast_id': dict['cast_id'], 'character': dict['character'], 'credit_id': dict['credit_id'], 'gender': dict['gender'],
                 'id': dict['id'], 'name': dict['name'], 'order': dict['order']}
        yield output

dfTest = pd.DataFrame(list_iterator(listTest))


In [12]:
dfTest

,cast_id,character,credit_id,gender,id,name,order
0,2,Max Rockatansky,52fe4933c3a368484e11f773,2,2524,Tom Hardy,0
1,35,Imperator Furiosa,560e8a2bc3a368681f00b80c,1,6885,Charlize Theron,1
2,4,Nux,52fe4933c3a368484e11f77b,2,3292,Nicholas Hoult,2
3,21,Immortan Joe,55549e64c3a3682086002b85,2,26060,Hugh Keays-Byrne,3
4,22,Slit,55549e84c3a36820800028e6,2,1056053,Josh Helman,4
5,14,Rictus Erectus,53d450f70e0a26283e002f36,2,24898,Nathan Jones,5
6,8,Toast the Knowing,52fe4933c3a368484e11f787,1,37153,Zoë Kravitz,6
7,12,The Splendid Angharad,53d450c50e0a262841002dc1,1,236048,Rosie Huntington-Whiteley,7
8,13,Capable,53d450e00e0a26284d002ba5,1,98522,Riley Keough,8
9,10,The Dag,52fe4933c3a368484e11f791,1,1036288,Abbey Lee,9


In [27]:
#¿Que pasa con las pelis que no tienen cast?!!! Ver algún ejemplo
dfTest = pd.DataFrame()

def cast_iterator():
    for id in dfMovies.index[0:10000]:
        for dict in dfMovies[dfMovies.index == id]['credits_cast'].iloc[0]:
            output = {'movie_id': id, 'cast_id': dict['cast_id'], 'character': dict['character'], 'credit_id': dict['credit_id'], 'gender': dict['gender'],
                     'id': dict['id'], 'name': dict['name'], 'order': dict['order']}
            yield output

dfTest = pd.DataFrame(cast_iterator())

tic=timeit.default_timer()
dfTest = pd.DataFrame(cast_iterator())
toc=timeit.default_timer()
toc - tic #elapsed time in seconds


8.223493980999592

In [28]:
dfTest.shape

(58965, 8)

In [31]:
dfTest[30:40]

,cast_id,character,credit_id,gender,id,movie_id,name,order
30,72,Owner (Ed's Drive-In),5905de84c3a3684ab401ea97,0,1807327,310307,Randall Taylor,14
31,8,Cheerleader,55528a68c3a3683fc80058b7,1,1466540,310307,Kabby Borders,15
32,9,Cheerleader #2,55528a79c3a3683fc80058be,0,1466541,310307,Valeri Rogers,16
33,1,Selim,5b3baeeec3a36859480007d4,2,76309,193424,Khaled Abol Naga,1
34,2,,5b3baef60e0a2623ff00079e,1,226438,193424,Hend Sabry,2
35,3,,5b3baefd92514102b800064b,2,140839,193424,Salah Abdallah,3
36,16,Himself,59230903c3a368723d071282,2,1556955,458674,Jim Smallman,0
37,1,Morgan Webster,592307ec925141485e0723cf,2,1566515,458674,Gavin Watkins,1
38,2,Himself,59230804c3a3687a8e06a34c,0,1773586,458674,James Drake,2
39,3,Herself,592308119251414ab106aac8,0,1582661,458674,Jinny,3


In [38]:
#¿Que pasa con las pelis que no tienen cast?!!! Ver algún ejemplo
dfTest = pd.DataFrame()

def cast_iterator():
    for id in dfMovies.index:
        if len(dfMovies[dfMovies.index == id]['credits_cast'].iloc[0])>0:
            for dict in dfMovies[dfMovies.index == id]['credits_cast'].iloc[0]:
                output = {'movie_id': id, 'cast_id': dict['cast_id'], 'character': dict['character'], 'credit_id': dict['credit_id'], 'gender': dict['gender'],
                         'id': dict['id'], 'name': dict['name'], 'order': dict['order']}
                yield output

dfTest = pd.DataFrame(cast_iterator())

tic=timeit.default_timer()
dfTest = pd.DataFrame(cast_iterator())
toc=timeit.default_timer()
toc - tic #elapsed time in seconds

546.2313306999995

In [39]:
dfTest.shape

(2278079, 8)

In [40]:
#No necesito controlar las pelis sin cast. El siguiente código es más rápido y da el mismo resultado.
dfTest = pd.DataFrame()

def cast_iterator():
    for id in dfMovies.index:
        for dict in dfMovies[dfMovies.index == id]['credits_cast'].iloc[0]:
            output = {'movie_id': id, 'cast_id': dict['cast_id'], 'character': dict['character'], 'credit_id': dict['credit_id'], 'gender': dict['gender'],
                     'id': dict['id'], 'name': dict['name'], 'order': dict['order']}
            yield output

dfTest = pd.DataFrame(cast_iterator())

tic=timeit.default_timer()
dfTest = pd.DataFrame(cast_iterator())
toc=timeit.default_timer()
toc - tic #elapsed time in seconds

335.6073752250013

In [36]:
dfTest.columns

Index(['cast_id', 'character', 'credit_id', 'gender', 'id', 'movie_id', 'name',
       'order'],
      dtype='object')

In [35]:
#¿Cómo me salen las pelis sin cast?
dfMovies[dfMovies['credits_cast'].map(len)==0].index[0:10]

Int64Index([493054, 310474, 308039, 340589, 358817, 290587, 528584, 149240,
            306100, 476407],
           dtype='int64', name='id')

In [37]:
dfTest[dfTest['movie_id']==493054]

,cast_id,character,credit_id,gender,id,movie_id,name,order


In [50]:
dfMovies[dfMovies.index ==76341]['credits_crew'].iloc[0][0].keys()

dict_keys(['credit_id', 'department', 'gender', 'id', 'job', 'name', 'profile_path'])

In [51]:
dfMovies[dfMovies.index ==76341]['credits_crew'].iloc[0][0]

{'credit_id': '53d451210e0a262841002dd1',
 'department': 'Writing',
 'gender': 2,
 'id': 1027146,
 'job': 'Writer',
 'name': 'Nick Lathouris',
 'profile_path': '/tlkHM8TUcKZFcsS7rGuShkCHeU.jpg'}

In [53]:
dfMovies[dfMovies.index ==76341]['genres'].iloc[0]

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 878, 'name': 'Science Fiction'},
 {'id': 53, 'name': 'Thriller'}]

In [56]:
np.sort(dfMovies['genres'].map(len).unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 11, 13])

In [54]:
dfMovies[dfMovies.index ==76341]['keywords'].iloc[0]

[{'id': 2964, 'name': 'future'},
 {'id': 3713, 'name': 'chase'},
 {'id': 4458, 'name': 'post-apocalyptic'},
 {'id': 4565, 'name': 'dystopia'},
 {'id': 5657, 'name': 'australia'},
 {'id': 10084, 'name': 'rescue'},
 {'id': 10349, 'name': 'survival'},
 {'id': 10562, 'name': 'on the run'},
 {'id': 155499, 'name': 'convoy'},
 {'id': 190954, 'name': 'peak oil'},
 {'id': 212516, 'name': 'dark future'}]

In [55]:
np.sort(dfMovies['keywords'].map(len).unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 54, 60,
       63, 80])

In [57]:
dfMovies[dfMovies.index ==76341]['production_companies'].iloc[0]

[{'id': 79,
  'logo_path': '/tpFpsqbleCzEE2p5EgvUq6ozfCA.png',
  'name': 'Village Roadshow Pictures',
  'origin_country': 'US'},
 {'id': 2537,
  'logo_path': None,
  'name': 'Kennedy Miller Productions',
  'origin_country': 'AU'},
 {'id': 174,
  'logo_path': '/ky0xOc5OrhzkZ1N6KyUxacfQsCk.png',
  'name': 'Warner Bros. Pictures',
  'origin_country': 'US'}]

In [58]:
np.sort(dfMovies['production_companies'].map(len).unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 25, 26])

In [59]:
dfMovies[dfMovies.index ==76341]['production_countries'].iloc[0]

[{'iso_3166_1': 'AU', 'name': 'Australia'},
 {'iso_3166_1': 'US', 'name': 'United States of America'}]

In [60]:
np.sort(dfMovies['production_countries'].map(len).unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 14, 15, 16, 20,
       25, 26])

In [61]:
dfMovies[dfMovies.index ==76341]['release_date'].iloc[0]

'2015-05-13'

In [62]:
dfMovies[dfMovies.index ==76341]['release_dates'].iloc[0]

[{'iso_3166_1': 'ES',
  'release_dates': [{'certification': '16',
    'iso_639_1': '',
    'note': '',
    'release_date': '2015-05-15T00:00:00.000Z',
    'type': 3}]},
 {'iso_3166_1': 'IN',
  'release_dates': [{'certification': 'A',
    'iso_639_1': '',
    'note': '',
    'release_date': '2015-05-15T00:00:00.000Z',
    'type': 3}]},
 {'iso_3166_1': 'ZA',
  'release_dates': [{'certification': '10-12PG V',
    'iso_639_1': '',
    'note': '',
    'release_date': '2015-05-15T00:00:00.000Z',
    'type': 3}]},
 {'iso_3166_1': 'MX',
  'release_dates': [{'certification': '',
    'iso_639_1': '',
    'note': '',
    'release_date': '2015-05-14T00:00:00.000Z',
    'type': 3}]},
 {'iso_3166_1': 'IL',
  'release_dates': [{'certification': '',
    'iso_639_1': '',
    'note': '',
    'release_date': '2015-05-14T00:00:00.000Z',
    'type': 3}]},
 {'iso_3166_1': 'BD',
  'release_dates': [{'certification': '',
    'iso_639_1': '',
    'note': '',
    'release_date': '2015-05-22T00:00:00.000Z',
    

In [14]:
dfTest[dfTest.index ==76341]['reviews_results'].iloc[0]

[{'author': 'Phileas Fogg',
  'content': "Fabulous action movie. Lots of interesting characters. They don't make many movies like this. The whole movie from start to finish was entertaining I'm looking forward to seeing it again. I definitely recommend seeing it.",
  'id': '55660928c3a3687ad7001db1',
  'url': 'https://www.themoviedb.org/review/55660928c3a3687ad7001db1'},
 {'author': 'Andres Gomez',
  'content': 'Good action movie with a decent script for the genre. The photography is really good too but, in the end, it is quite repeating itself from beginning to end and the stormy OST is exhausting.',
  'id': '55732a53925141456e000639',
  'url': 'https://www.themoviedb.org/review/55732a53925141456e000639'},
 {'author': 'extoix',
  'content': 'Awesome movie!  WITNESS ME will stick with me forever!',
  'id': '55edd26792514106d600e380',
  'url': 'https://www.themoviedb.org/review/55edd26792514106d600e380'},
 {'author': 'Cineport',
  'content': 'Fantastic action that makes up for some plot

In [27]:
dfMovies.columns

Index(['belongs_to_collection_id', 'belongs_to_collection_name', 'budget',
       'credits_cast', 'credits_crew', 'genres', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'release_dates', 'revenue', 'reviews_page', 'reviews_results',
       'reviews_total_pages', 'reviews_total_results', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'titles',
       'vote_average', 'vote_count'],
      dtype='object')

In [0]:
#*********************************

In [0]:
dfMovies.to_csv('movies.csv')

In [0]:
#Copio el csv de Colab a mi drive. Y después, desde Drive, ya los puedo bajar a local.
!cp movies.csv /content/drive/"My Drive"

In [76]:
!ls "/content/drive/My Drive/" -lh

total 4.2G
drwx------ 2 root root  4.0K Jul 14 07:08  20180714_Deep_Learning
drwx------ 2 root root  4.0K Jun 26  2016 'avast! Files (XT1068, UWbYzWPVmAmwlrlD)'
drwx------ 2 root root  4.0K Jul 18 18:38 'Colab Notebooks'
-rw------- 1 root root   44K Feb  4  2018  contacts.vcf
-rw------- 1 root root     1 Apr  9  2018  facturaIVA.gsheet
-rw------- 1 root root     1 Oct 11 07:38 'Mis cuentas.gsheet'
-rw------- 1 root root 1019M Oct 14 16:41  movies.csv
-rw------- 1 root root  1.5G Oct 12 15:09  movies.zip
-rw------- 1 root root     1 Oct 14 08:57 'Nuestras Cuentas.gsheet'
-rw------- 1 root root  1.8G Sep 24 21:07  people.zip
-rw------- 1 root root     1 Jul 24 18:17 'Propuesta de una idea.gslides'
drwx------ 2 root root  4.0K Apr  7  2015  Screencastify
-rw------- 1 root root     1 Sep 30  2017  Series.gdoc
-rw------- 1 root root     1 Jan  2  2016 'Trabajos Iñigo.gsheet'
-rw------- 1 root root   31M Sep 25 21:00  trial.zip
-rw------- 1 root root  2.7K Oct 14 11:04  typesCheck.csv
-rw--

In [0]:
def data_iterator(path):
    for root, dirs, files in os.walk(path):
        for f in files:
            if f.endswith('.json'):
                fp = os.path.join(root,f)
                with open(fp) as o:
                    data = ujson.load(o)
                yield data


df2 = pd.DataFrame(data_iterator(path))
#df.set_index('track_id', inplace=True)